In [17]:
#imports

import os
import glob
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
from osgeo import gdal
from osgeo import gdalconst
import numpy as np
import rasterio
import matplotlib.pyplot as plt
import json

In [23]:
# download Sentinel-2 data by date and bounding box
# lat_max = top_value
# lat_min = bottom_value
# lon_max = right_value
# lon_min = left_value


def download_sentinel_data(username, password, start_date, end_date, bounding_box, output_directory):
    """
    Downloads Sentinel-2 data using the Copernicus API Hub.

    Args:
        username (str): Your Copernicus Open Access Hub username.
        password (str): Your Copernicus Open Access Hub password.
        start_date (str): Start date in 'YYYYMMDD' format (e.g., '20220101').
        end_date (str): End date in 'YYYYMMDD' format (e.g., '20220131').
        bounding_box (list): Bounding box coordinates in the format [lon_min, lat_min, lon_max, lat_max].
        output_directory (str): Directory where downloaded data will be stored.

    Returns:
        None
    """
    # Initialize the Sentinel API with your credentials
    api = SentinelAPI(username, password, 'https://apihub.copernicus.eu/dhus')

    # Convert the bounding box to Well-Known Text (WKT) format
    footprint = geojson_to_wkt({
        "type": "Polygon",
        "coordinates": [bounding_box]
    })

    # Search for Sentinel-2 products within the specified date range and bounding box
    products = api.query(footprint,
                         date=(start_date, end_date),
                         platformname='Sentinel-2',
                         cloudcoverpercentage=(0, 30))  # You can adjust the cloud cover percentage as needed

    # Download the products
    for product_id, product_info in products.items():
        print(f"Downloading product {product_id}...")
        api.download(product_id, directory_path=output_directory)

if __name__ == "__main__":
    # Set your Copernicus Open Access Hub credentials
    username = "lijo8146@colorado.edu"
    password = "m@G413Fly393"

    # Define the date range and bounding box for your area of interest
    start_date = "20220101"
    end_date = "20220131"
    bounding_box = [-120.529210, 33.183798, -114.131493, 38.405490]  # bounding box coordinates projected in NAD 1983
    output_directory = r"C:\Users\lilly\Documents\Sentinel"  # desired output directory

In [11]:
# using the API with a polygon bounding box


from sentinelsat import SentinelAPI, geojson_to_wkt

def download_sentinel_data(username, password, start_date, end_date, polygon, output_directory):
    """
    Downloads Sentinel-2 data using the Copernicus API Hub.

    Args:
        username (str): Your Copernicus Open Access Hub username.
        password (str): Your Copernicus Open Access Hub password.
        start_date (str): Start date in 'YYYYMMDD' format (e.g., '20220101').
        end_date (str): End date in 'YYYYMMDD' format (e.g., '20220131').
        polygon (dict): GeoJSON polygon defining the area of interest.
        output_directory (str): Directory where downloaded data will be stored.

    Returns:
        None
    """
    # Initialize the Sentinel API with your credentials
    api = SentinelAPI(username, password, 'https://apihub.copernicus.eu/dhus')

    # Convert the GeoJSON polygon to Well-Known Text (WKT) format
    footprint = geojson_to_wkt(polygon)

    # Search for Sentinel-2 products within the specified date range and bounding box
    products = api.query(footprint,
                         date=(start_date, end_date),
                         platformname='Sentinel-2',
                         cloudcoverpercentage=(0, 30))  # You can adjust the cloud cover percentage as needed

    # Download the products
    for product_id, product_info in products.items():
        print(f"Downloading product {product_id}...")
        api.download(product_id, directory_path=output_directory)

if __name__ == "__main__":
    # Set your Copernicus Open Access Hub credentials
    username = "lijo8146@colorado.edu"
    password = "m@G413Fly393"

    # Define the date range and bounding box for your area of interest as a GeoJSON polygon
    polygon = {
        "type": "Polygon",
        "coordinates": [
            [
                [lon_min, lat_min],
                [lon_max, lat_min],
                [lon_max, lat_max],
                [lon_min, lat_max],
                [lon_min, lat_min]
            ]
        ]
    }

    start_date = "20220101"
    end_date = "20220131"
    output_directory = "path_to_output_directory"  # Replace with your desired output directory  


NameError: name 'lon_min' is not defined

In [13]:
 # Call the download_sentinel_data function
download_sentinel_data(username, password, start_date, end_date, polygon, output_directory)

NameError: name 'polygon' is not defined

In [ ]:
# downscale layers to 10m resolution


def convert_sentinel_bands_to_10m_resolution(input_directory, output_directory):
    """
    Converts Sentinel-2 bands to 10-meter resolution.

    Args:
        input_directory (str): Directory containing Sentinel-2 data.
        output_directory (str): Directory where converted data will be stored.

    Returns:
        None
    """
    # List all Sentinel-2 band files in the input directory
    band_files = glob.glob(os.path.join(input_directory, 'B*.jp2'))

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    for band_file in band_files:
        # Define the output file path
        output_file = os.path.join(output_directory, os.path.basename(band_file))

        # Open the input band file
        ds = gdal.Open(band_file, gdalconst.GA_ReadOnly)

        # Perform resampling and reprojection to 10-meter resolution
        gdal.Warp(output_file, ds, xRes=10, yRes=10, resampleAlg=gdalconst.GRA_Bilinear)

        # Close the input dataset
        ds = None

if __name__ == "__main__":
    # Define input and output directories
    input_directory = "path_to_input_directory"  # Replace with the directory containing Sentinel-2 data
    output_directory = "path_to_output_directory"  # Replace with your desired output directory

    # Call the convert_sentinel_bands_to_10m_resolution function
    convert_sentinel_bands_to_10m_resolution(input_directory, output_directory)

# mask out clouds and water using the SCL attribute of Sentinel data
# mask out clouds and water using the SCL (Scene Classification Layer) attribute of Sentinel-2 data, you can create a function that reads the SCL band, applies a mask to identify cloudy and water areas, and then applies this mask to the other bands. 


def mask_clouds_and_water(input_directory, output_directory):
    """
    Masks out clouds and water in Sentinel-2 data using the SCL band.

    Args:
        input_directory (str): Directory containing Sentinel-2 data.
        output_directory (str): Directory where masked data will be stored.

    Returns:
        None
    """
    # List all Sentinel-2 band files in the input directory
    band_files = glob.glob(os.path.join(input_directory, 'B*.jp2'))

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Determine the SCL file path (assuming it's in the same directory)
    scl_file = os.path.join(input_directory, 'SCL.jp2')

    for band_file in band_files:
        # Define the output file path
        output_file = os.path.join(output_directory, os.path.basename(band_file))

        # Open the input band and SCL files
        ds_band = gdal.Open(band_file, gdalconst.GA_ReadOnly)
        ds_scl = gdal.Open(scl_file, gdalconst.GA_ReadOnly)

        # Read band data as a numpy array
        band_data = ds_band.ReadAsArray()

        # Read SCL data as a numpy array
        scl_data = ds_scl.ReadAsArray()

        # Create a mask for cloudy and water pixels
        # In the SCL band, value 3 corresponds to cloud and value 6 corresponds to water
        cloud_water_mask = np.logical_or(scl_data == 3, scl_data == 6)

        # Apply the mask to the band data by setting masked values to a nodata value (e.g., 0)
        band_data[cloud_water_mask] = 0

        # Create an output GeoTIFF file with the same metadata as the input band
        driver = gdal.GetDriverByName("GTiff")
        output_ds = driver.Create(output_file, ds_band.RasterXSize, ds_band.RasterYSize, 1, gdalconst.GDT_Float32)
        output_ds.SetGeoTransform(ds_band.GetGeoTransform())
        output_ds.SetProjection(ds_band.GetProjection())
        output_ds.GetRasterBand(1).WriteArray(band_data)

        # Close the datasets
        ds_band = None
        ds_scl = None
        output_ds = None

if __name__ == "__main__":
    # Define input and output directories
    input_directory = "path_to_input_directory"  # Replace with the directory containing Sentinel-2 data
    output_directory = "path_to_output_directory"  # Replace with your desired output directory

    # Call the mask_clouds_and_water function
    mask_clouds_and_water(input_directory, output_directory)

In [ ]:
# calculate NDVI using bands 4 and 8
# calculate the Normalized Difference Vegetation Index (NDVI) from Sentinel-2 bands 4 (red) and 8 (nir), you can create a Python function. NDVI is calculated as (NIR - Red) / (NIR + Red).


def calculate_ndvi(input_directory, output_directory):
    """
    Calculates NDVI from Sentinel-2 bands 4 (red) and 8 (NIR).

    Args:
        input_directory (str): Directory containing Sentinel-2 data.
        output_directory (str): Directory where NDVI data will be stored.

    Returns:
        None
    """
    # List all Sentinel-2 band files in the input directory
    band4_file = glob.glob(os.path.join(input_directory, 'B04.jp2'))[0]  # Red band
    band8_file = glob.glob(os.path.join(input_directory, 'B08.jp2'))[0]  # NIR band

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    for band_file in [band4_file, band8_file]:
        # Define the output file path
        output_file = os.path.join(output_directory, os.path.basename(band_file).replace('B0', 'NDVI'))

        # Open the input band file
        ds_band = gdal.Open(band_file, gdalconst.GA_ReadOnly)

        # Read band data as a numpy array
        band_data = ds_band.ReadAsArray()

        # Calculate NDVI
        if 'B04' in band_file:
            red_band = band_data.astype(np.float32)
        elif 'B08' in band_file:
            nir_band = band_data.astype(np.float32)

    # Calculate NDVI
    ndvi = (nir_band - red_band) / (nir_band + red_band)

    # Create an output GeoTIFF file with the same metadata as the input band
    driver = gdal.GetDriverByName("GTiff")
    output_ds = driver.Create(output_file, ds_band.RasterXSize, ds_band.RasterYSize, 1, gdalconst.GDT_Float32)
    output_ds.SetGeoTransform(ds_band.GetGeoTransform())
    output_ds.SetProjection(ds_band.GetProjection())
    output_ds.GetRasterBand(1).WriteArray(ndvi)

    # Close the datasets
    ds_band = None
    output_ds = None

if __name__ == "__main__":
    # Define input and output directories
    input_directory = "path_to_input_directory"  # Replace with the directory containing Sentinel-2 data
    output_directory = "path_to_output_directory"  # Replace with your desired output directory

    # Call the calculate_ndvi function
    calculate_ndvi(input_directory, output_directory)

In [ ]:
# calculate mean NDVI for all time-steps


def calculate_ndvi(input_directory, output_directory):
    """
    Calculates NDVI from Sentinel-2 bands 4 (red) and 8 (NIR).

    Args:
        input_directory (str): Directory containing Sentinel-2 data for multiple time-steps.
        output_directory (str): Directory where mean NDVI data will be stored.

    Returns:
        None
    """
    # List all time-steps (subdirectories) in the input directory
    time_steps = [d for d in os.listdir(input_directory) if os.path.isdir(os.path.join(input_directory, d))]

    # Create the output directory if it doesn't exist
    os.makedirs(output_directory, exist_ok=True)

    # Initialize arrays to store NDVI values and count of valid pixels
    ndvi_sum = None
    valid_pixel_count = None

    for time_step in time_steps:
        # Get the directory containing Sentinel-2 data for the current time-step
        time_step_directory = os.path.join(input_directory, time_step)

        # List all Sentinel-2 band files in the current time-step directory
        band4_file = glob.glob(os.path.join(time_step_directory, 'B04.jp2'))[0]  # Red band
        band8_file = glob.glob(os.path.join(time_step_directory, 'B08.jp2'))[0]  # NIR band

        for band_file in [band4_file, band8_file]:
            # Open the input band file
            ds_band = gdal.Open(band_file, gdalconst.GA_ReadOnly)

            # Read band data as a numpy array
            band_data = ds_band.ReadAsArray()

            # Calculate NDVI
            if 'B04' in band_file:
                red_band = band_data.astype(np.float32)
            elif 'B08' in band_file:
                nir_band = band_data.astype(np.float32)

        # Calculate NDVI for the current time-step
        ndvi = (nir_band - red_band) / (nir_band + red_band)

        # Initialize the arrays for NDVI sum and valid pixel count
        if ndvi_sum is None:
            ndvi_sum = np.zeros_like(ndvi, dtype=np.float64)
            valid_pixel_count = np.zeros_like(ndvi, dtype=np.int64)

        # Update the sum of NDVI values and valid pixel count
        ndvi_sum += ndvi
        valid_pixel_count += 1

    # Calculate mean NDVI by dividing the sum by the valid pixel count
    mean_ndvi = np.divide(ndvi_sum, valid_pixel_count, where=valid_pixel_count != 0)

    # Get the metadata (geotransform and projection) from one of the input bands
    ds_band = gdal.Open(band4_file, gdalconst.GA_ReadOnly)
    geotransform = ds_band.GetGeoTransform()
    projection = ds_band.GetProjection()
    ds_band = None

    # Create an output GeoTIFF file with the same metadata as the input band
    output_file = os.path.join(output_directory, 'mean_ndvi.tif')
    driver = gdal.GetDriverByName("GTiff")
    output_ds = driver.Create(output_file, mean_ndvi.shape[1], mean_ndvi.shape[0], 1, gdalconst.GDT_Float32)
    output_ds.SetGeoTransform(geotransform)
    output_ds.SetProjection(projection)
    output_ds.GetRasterBand(1).WriteArray(mean_ndvi)

    # Close the output dataset
    output_ds = None

if __name__ == "__main__":
    # Define input and output directories
    input_directory = "path_to_input_directory"  # Replace with the directory containing Sentinel-2 data for multiple time-steps
    output_directory = "path_to_output_directory"  # Replace with your desired output directory

    # Call the calculate_ndvi function
    calculate_ndvi(input_directory, output_directory)

In [ ]:
# plot mean NDVI


def plot_mean_ndvi(ndvi_file):
    """
    Plots the mean NDVI from a GeoTIFF file.

    Args:
        ndvi_file (str): Path to the GeoTIFF file containing mean NDVI data.

    Returns:
        None
    """
    # Open the GeoTIFF file using rasterio
    with rasterio.open(ndvi_file) as src:
        # Read the NDVI data as a numpy array
        ndvi_data = src.read(1)

        # Create a colormap for NDVI (green to red)
        cmap = plt.cm.RdYlGn

        # Set nodata values to transparent
        cmap.set_bad(alpha=0)

        # Create a plot
        plt.figure(figsize=(10, 10))
        plt.imshow(ndvi_data, cmap=cmap, vmin=-1, vmax=1)
        plt.colorbar(label='NDVI')
        plt.title('Mean NDVI')
        plt.axis('off')

        # Show the plot
        plt.show()

if __name__ == "__main__":
    # Define the path to the mean NDVI GeoTIFF file
    ndvi_file = "path_to_mean_ndvi.tif"  # Replace with the actual file path

    # Call the plot_mean_ndvi function
    plot_mean_ndvi(ndvi_file)

In [ ]:
# plot spectral signature for all bands reads and plots the pixel values for each band


def plot_spectral_signature(input_directory):
    """
    Plots the spectral signature from all bands of Sentinel-2 data.

    Args:
        input_directory (str): Directory containing Sentinel-2 data.

    Returns:
        None
    """
    # List all Sentinel-2 band files in the input directory
    band_files = sorted(glob.glob(os.path.join(input_directory, 'B*.jp2')))

    # Initialize an empty list to store band names
    band_names = []

    # Initialize empty arrays to store spectral data for each band
    spectral_data = []

    # Open and read data for each band
    for band_file in band_files:
        # Get the band name from the file name (e.g., B02.jp2)
        band_name = os.path.splitext(os.path.basename(band_file))[0]
        band_names.append(band_name)

        # Open the band file using GDAL
        ds_band = gdal.Open(band_file, gdal.GA_ReadOnly)

        # Read the band data as a numpy array
        band_data = ds_band.ReadAsArray()

        # Append the band data to the spectral_data list
        spectral_data.append(band_data)

    # Create a plot for the spectral signature
    plt.figure(figsize=(12, 6))
    for band_name, band_data in zip(band_names, spectral_data):
        plt.plot(band_data, label=band_name)

    # Set plot labels and title
    plt.xlabel('Pixel Value')
    plt.ylabel('Reflectance')
    plt.title('Spectral Signature of Sentinel-2 Bands')
    plt.legend()
    plt.grid(True)

    # Show the plot
    plt.show()

if __name__ == "__main__":
    # Define input directory containing Sentinel-2 data
    input_directory = "path_to_input_directory"  # Replace with the directory containing Sentinel-2 data

    # Call the plot_spectral_signature function
    plot_spectral_signature(input_directory)